In [1]:
# Standard packages
import tempfile
import warnings
import urllib
import shutil
import os
# Less standard, but still pip- or conda-installable
import pandas as pd
import numpy as np
import matplotlib.pyplot as plt
import matplotlib
import rasterio
import re
import rtree
import shapely
import pickle
import data_eng.az_proc as ap

In [2]:
import PIL
print('PIL',PIL.__version__)
from PIL import Image
Image.MAX_IMAGE_PIXELS = None

PIL 8.2.0


In [3]:
parent_directory = "C:\chip_allocation_temp"
annotation_directory = "Jonathan"
number_of_tiles = 20
tiles_remaining = "tile_name_tile_url_remaining_expanded.npy"
tiles_labeled = "tile_name_tile_url_labeled.npy"

In [4]:
dist = ap.annotator(annotation_directory) #create the processing class
dist.state_dcc_directory(parent_directory)
dist.number_of_tiles(number_of_tiles)
dist.get_tile_urls(tiles_remaining)
dist.make_subdirectories()

Unlabeled Tiles (1128, 2)


In [67]:
#np.load('tile_name_tile_url_remaining_expanded.npy').shape
#row_to_remove = 2
#error_downloading = np.concatenate((np.load('tile_name_tile_url_error_downloading.npy'), dist.tile_name_tile_url_tiles_for_annotators[row_to_remove]))
#np.save('tile_name_tile_url_error_downloading.npy', error_downloading)
# Remove the rows with errors
#np.save('tile_name_tile_url_remaining_expanded',dist.tile_name_tile_url_unlabeled)
#np.save('tile_name_tile_url_remaining_expanded',np.delete(np.load(tiles_remaining), row_to_remove, axis = 0))
#np.load('tile_name_tile_url_remaining_expanded.npy').shape

(1128, 2)

In [70]:
dist.download_images()
dist.tile_rename()

Bypassing download of already-downloaded file m_4108723_se_16_060_20180706_20181211.tif
Bypassing download of already-downloaded file m_2909837_ne_14_060_20181102_20190518.tif


100% (586766746 of 586766746) |##########| Elapsed Time: 0:00:43 Time:  0:00:43


...done, 586766746 bytes.


100% (567348363 of 567348363) |##########| Elapsed Time: 0:00:44 Time:  0:00:44
N/A% (0 of 611471072) |                  | Elapsed Time: 0:00:00 ETA:  --:--:--

...done, 567348363 bytes.


...

100% (577777440 of 577777440) |##########| Elapsed Time: 0:00:16 Time:  0:00:16


...done, 577777440 bytes.


100% (454456857 of 454456857) |##########| Elapsed Time: 0:00:17 Time:  0:00:17


...done, 454456857 bytes.


['C:\\chip_allocation_temp/Jonathan\\tiles\\https_naipblobs.blob.core.windows.net_naip_v002_in_2018_in_060cm_2018_41087_m_4108723_se_16_060_20180706_20181211.tif',
 'C:\\chip_allocation_temp/Jonathan\\tiles\\https_naipblobs.blob.core.windows.net_naip_v002_tx_2018_tx_060cm_2018_29098_m_2909837_ne_14_060_20181102_20190518.tif',
 'C:\\chip_allocation_temp/Jonathan\\tiles\\https_naipblobs.blob.core.windows.net_naip_v002_in_2018_in_060cm_2018_41086_m_4108615_sw_16_060_20180708_20181211.tif',
 'C:\\chip_allocation_temp/Jonathan\\tiles\\https_naipblobs.blob.core.windows.net_naip_v002_tx_2018_tx_060cm_2018_30100_m_3010043_se_14_060_20180623_20190422.tif',
 'C:\\chip_allocation_temp/Jonathan\\tiles\\https_naipblobs.blob.core.windows.net_naip_v002_tx_2018_tx_060cm_2018_26098_m_2609821_sw_14_060_20190105_20190517.tif',
 'C:\\chip_allocation_temp/Jonathan\\tiles\\https_naipblobs.blob.core.windows.net_naip_v002_wi_2018_wi_060cm_2018_43091_m_4309114_ne_15_060_20180909.tif',
 'C:\\chip_allocation_tem

In [ ]:
dist.chip_tiles()

In [4]:
tile_names = os.listdir(dist.tiles_dir) #get a list of all of the tiles in tiles directory

In [1]:
for tile_name in tile_names: #index over the tiles in the tiles_dir 

    file_name, ext = os.path.splitext(tile_name) # File name
    print(tile_name)
    tile = Image.open(os.path.join(dist.tiles_dir, tile_name)) #open image
    tile = tile.convert('RGB') #to file issues

NameError: name 'tile_names' is not defined

In [ ]:
        
for tile_name in tile_names: #index over the tiles in the tiles_dir 

    file_name, ext = os.path.splitext(tile_name) # File name
    print(tile_name)

    tile = Image.open(os.path.join(self.tiles_dir, tile_name)) #open image
    tile = tile.convert('RGB') #to file issues

    width, height = tile.size
    item_width = int(512)
    col = int(width/512)+1
    row = int(height/512)+1
    box_list = []

    count = 1
    for j in range(0,col):
        for i in range(0,row):
            box = (i*item_width, j*item_width, (i+1)*item_width, (j+1)*item_width)

            chips_save_path = os.path.join(self.chips_dir, file_name+ '_'+ \
                                           str(count).zfill(6) + '.jpg') 
                # The index is a six-digit number like '000023'. 

            #chip = tile.crop(box)
            #chip = chip.save(chips_save_path)
            chip = tile.crop(box).save(chips_save_path)
            count += 1
    print(count)

In [20]:
dist.track_tile_annotations(tiles_labeled)
np.save('tile_name_tile_url_remaining_expanded', dist.tile_name_tile_url_remaining)
np.save('tile_name_tile_url_labeled', dist.tile_name_tile_url_labeled)